## 1- Data Ingestion
Source : https://covidtracking.com/data/api

Data of covid 19 cases in USA

In [ ]:
#test commit
import requests
import csv
from pyspark.sql import SparkSession
from pyspark.sql.types import *

url = 'https://api.covidtracking.com/v1/states/daily.csv'
response = requests.get(url)

csv_data = csv.reader(response.text.strip().split('\n'))

header = next(csv_data)

fields = [StructField(field_name, StringType(), True) for field_name in header]
schema = StructType(fields)

spark = SparkSession.builder.appName("COVID Tracking Data - Data Ingestion").getOrCreate()

df = spark.createDataFrame(csv_data, schema=schema)
destination_path = '/FileStore/tables/Covid-USA/raw/daily_cases_raw'
df.write.mode('overwrite').format("parquet").save(destination_path)
df.show(10)


+--------+-----+--------+-------------+--------+-------+----------------------+----------------+---------------------+----------------------+--------------+---------------+---------------------+----------------------+---------+---------------+--------------------+-----------+-----+------------+----------------------+--------------------+---------------+------------------+------------------+------------------+--------------+-------------+------------------------+---------------------+------------------+---------------------+---------------------+------------------------+---------------------------+---------------------------+-----------------------+--------------------------+-----------------+--------------------+----+----------------+----------------+-------+------------------------+-------+----------------+-------------+--------------------+--------------------+---------------+--------------------+-------------+-------------+-----+-----+
|    date|state|positive|probableCases|negative

## 2- Data clean and standardize

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import when, regexp_replace, col, trim, to_date

destination_path = '/FileStore/tables/Covid-USA/raw/daily_cases_raw'
spark = SparkSession.builder.appName("COVID Tracking Data - Data cleaning").getOrCreate()
df = spark.read.format("parquet").load(destination_path)
df = df.select("date","state", "positive", "death", "totalTestsViral")

df = df.withColumn("state", trim(df.state))


df = df.withColumn("positive", when(col("positive").isNull(), 0).otherwise(col("positive")))
df = df.withColumn("death", when(col("death").isNull(), 0).otherwise(col("death")))

average_tests = df.selectExpr("avg(totalTestsViral)").collect()[0][0]
df = df.withColumn("totalTestsViral", when(col("totalTestsViral").isNull(), average_tests).otherwise(col("totalTestsViral")))
df = df.withColumn('date', to_date(df['date'], 'yyyyMMdd'))

destination_path = '/FileStore/tables/Covid-USA/bronze/daily_cases_bronze'
df.write.mode('overwrite').format("parquet").save(destination_path)
df.show(10)


+----------+-----+--------+-----+---------------+
|      date|state|positive|death|totalTestsViral|
+----------+-----+--------+-----+---------------+
|2021-01-30|   NE|  190570| 1920|        2090962|
|2021-01-30|   NH|   65362| 1042|        1254902|
|2021-01-30|   NJ|  692543|21455|        9326347|
|2021-01-30|   NM|  173539| 3265|        2328334|
|2021-01-30|   NV|  277349| 4264|               |
|2021-01-30|   NY| 1399863|35036|               |
|2021-01-30|   OH|  892781|11121|        9033143|
|2021-01-30|   OK|  386590| 3504|        3166000|
|2021-01-30|   OR|  141729| 1938|        3162750|
|2021-01-30|   PA|  839239|21602|               |
+----------+-----+--------+-----+---------------+
only showing top 10 rows



## 3- Data preparation

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

destination_path = '/FileStore/tables/Covid-USA/bronze/daily_cases_bronze'
spark = SparkSession.builder.appName("COVID Tracking Data - Data prep").getOrCreate()
df = spark.read.format("parquet").load(destination_path)

state_dict = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota',
    'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina',
    'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania',
    'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas',
    'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming'
}
state_cond = [when(col('state') == k, v).otherwise(None).alias(v) for k, v in state_dict.items()]

df = df.select(['date', *state_cond, col('positive'), col('death'), col('totalTestsViral')])
df = df.withColumn('state', coalesce(*state_dict.values())) 
df = df.drop(*state_dict.keys())

df = df.withColumn("year", year("date"))
df_filtered = df.filter(col('state').isNotNull())
df_grouped = df_filtered.groupBy("state", "year") \
               .agg(sum("positive").alias("total_positive_cases"), 
                    sum("death").alias("total_deaths"))
df_ordered = df_grouped.orderBy("state", "year")
df_ordered = df_ordered.withColumn("total_positive_cases", col("total_positive_cases").cast("integer"))
df_ordered = df_ordered.withColumn("total_deaths", col("total_deaths").cast("integer"))

destination_path = '/FileStore/tables/Covid-USA/silver/daily_cases_silver'
df_ordered.write.mode('overwrite').format("parquet").save(destination_path)
df_ordered.show(10)

+----------+----+--------------------+------------+
|     state|year|total_positive_cases|total_deaths|
+----------+----+--------------------+------------+
|   Alabama|2020|            32225868|      526142|
|   Alabama|2021|            29836458|      512122|
|    Alaska|2020|             2695918|       14079|
|    Alaska|2021|             3462229|       17407|
|   Arizona|2020|            47068180|     1047790|
|   Arizona|2021|            48396191|      870644|
|  Arkansas|2020|            18390160|      285482|
|  Arkansas|2021|            19232376|      316079|
|California|2020|           171681857|     3039476|
|California|2021|           207418602|     2693613|
+----------+----+--------------------+------------+
only showing top 10 rows



## 4- Data Consuming and Visualization

In [ ]:
import folium
from pyspark.sql import SparkSession
import pandas as pd

destination_path = '/FileStore/tables/Covid-USA/silver/daily_cases_silver'
spark = SparkSession.builder.appName("COVID Tracking Data - Data Visualization").getOrCreate()
df = spark.read.format("parquet").load(destination_path)
df_pandas = df.toPandas()

us_states_geojson = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'

m = folium.Map(location=[37, -102], zoom_start=4)
folium.Choropleth(
    geo_data=us_states_geojson,
    name='Total Positive Cases',
    data=df_pandas,
    columns=['state', 'total_positive_cases'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight=True,
    legend_name='Total Positive Cases'
).add_to(m)

folium.Choropleth(
    geo_data=us_states_geojson,
    name='Total Deaths',
    data=df_pandas,
    columns=['state', 'total_deaths'],
    key_on='feature.properties.name',
    fill_color='PuBuGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight=True,
    legend_name='Total Deaths'
).add_to(m)

# Add a layer control to the map
folium.LayerControl().add_to(m)

# Display the map
m

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 
 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>
 
 <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
 <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-1.12.4.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_34234f02f0b82f611201aa4311d49999 {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>
 
 <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
</head>
<body>
 
 
 <div class="folium-map" id="map_34234f02f0b82f611201aa4311d49999" ></div>
 
</body>
<script>
 
 
 var map_34234f02f0b82f611201aa4311d49999 = L.map(
 "map_34234f02f0b82f611201aa4311d49999",
 {
 center: [37.0, -102.0],
 crs: L.CRS.EPSG3857,
 zoom: 4,
 zoomControl: true,
 preferCanvas: false,
 }
 );

 

 
 
 var tile_layer_d5f024b1520fd37b230c9e79f2b3ce59 = L.tileLayer(
 "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
 {"attribution": "Data by \u0026copy; \u003ca target=\"_blank\" href=\"http://openstreetmap.org\"\u003eOpenStreetMap\u003c/a\u003e, under \u003ca target=\"_blank\" href=\"http://www.openstreetmap.org/copyright\"\u003eODbL\u003c/a\u003e.", "detectRetina": false, "maxNativeZoom": 18, "maxZoom": 18, "minZoom": 0, "noWrap": false, "opacity": 1, "subdomains": "abc", "tms": false}
 ).addTo(map_34234f02f0b82f611201aa4311d49999);
 
 
 var choropleth_11e8fc37925b43f0f6b6fa535a074f7e = L.featureGroup(
 {}
 ).addTo(map_34234f02f0b82f611201aa4311d49999);
 
 
 function geo_json_b00aa0bcd52956e39daa35e382e3a3ec_styler(feature) {
 switch(feature.id) {
 case "AZ": case "GA": case "IN": case "MI": case "NJ": case "NC": case "OH": case "PA": case "TN": case "WI": 
 return {"color": "black", "fillColor": "#fed976", "fillOpacity": 0.7, "opacity": 0.2, "weight": 1};
 case "CA": 
 return {"color": "black", "fillColor": "#bd0026", "fillOpacity": 0.7, "opacity": 0.2, "weight": 1};
 case "FL": 
 return {"color": "black", "fillColor": "#fd8d3c", "fillOpacity": 0.7, "opacity": 0.2, "weight": 1};
 case "IL": case "NY": 
 return {"color": "black", "fillColor": "#feb24c", "fillOpacity": 0.7, "opacity": 0.2, "weight": 1};
 case "TX": 
 return {"color": "black", "fillColor": "#f03b20", "fillOpacity": 0.7, "opacity": 0.2, "weight": 1};
 default:
 return {"color": "black", "fillColor": "#ffffb2", "fillOpacity": 0.7, "opacity": 0.2, "weight": 1};
 }
 }
 function geo_json_b00aa0bcd52956e39daa35e382e3a3ec_highlighter(feature) {
 switch(feature.id) {
 default:
 return {"fillOpacity": 0.8999999999999999, "weight": 3};
 }
 }

 function geo_json_b00aa0bcd52956e39daa35e382e3a3ec_onEachFeature(feature, layer) {
 layer.on({
 mous